**PERMASALAHAN**

Perusahaan menginginkan sebuah yang efektif sehingga mampu menarik respon dari banyak user untuk mengisi kuesioner. Perusahaan memiliki dua pilihan yaitu menggunakan iklan biasa (gambar) atau menggunakan iklan interaktif (video). Perusahaan telah melakukan eksperimen dan mengumpulkan data dengan membagi menjadi 2 kelompok user :
*   Control : user yang diperlihatkan iklan biasa
*   Exposed : user yang diperlihatkan iklan interaktif

Perusahaan ingin mengetahui tingkat kesuksesan dari dua jenis iklan tersebut.



**Import library**

In [2]:
from datetime import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#ambil data
dataku = pd.read_csv("AdSmartABdata.csv")

In [4]:
#liat sampel data
dataku.sample(6)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
5224,a59a7d65-a59f-4bf3-a5a6-347f33ec3675,exposed,2020-07-10,2,Generic Smartphone,6,Chrome Mobile,0,0
7414,ea8abad9-e3ae-436c-8945-d8e608355ef7,exposed,2020-07-09,8,Generic Smartphone,6,Chrome Mobile,0,0
3974,7f966c74-0058-43e6-bc7c-b9649c8df0c2,exposed,2020-07-04,11,Generic Smartphone,6,Chrome Mobile WebView,0,0
4827,99f8f9a2-7950-4203-9129-89d6c26a5d75,exposed,2020-07-09,16,Samsung SM-G950F,6,Chrome Mobile WebView,0,0
4184,8664254f-300f-430d-ba11-4a3516a70ecf,exposed,2020-07-09,1,Generic Smartphone,6,Chrome Mobile,0,0
6266,c75246b8-6c22-4745-a840-9b296fa3f04f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


**Data Preparation**

In [5]:
#lihat banyak user
dataku.shape

(8077, 9)

In [6]:
#lihat missing value
dataku.isnull().values.any()

False

In [7]:
#lihat apakah ada data yang nilai yes dan no = 1
dataku[(dataku['yes'] == 1) & (dataku['no'] == 1)].shape[0]

0

In [8]:
#lihat apakah ada data yang nilai yes dan no = 0
dataku[(dataku['yes'] == 0) & (dataku['no'] == 0)].shape[0]

6834

Terdapat 6834 user yang tidak memberikan respon

In [9]:
#lihat berapa persen user yang mengabaikan iklan dari total user 8077
terabaikan = (100 * dataku[(dataku['yes'] == 0) & (dataku['no'] == 0)].shape[0] / dataku.shape[0])
print(f'{terabaikan}%')

84.61062275597375%


Jumlah user yang tidak memberikan respon sebanyak 84.6% dari total jumlah user yang mendapatkan iklan

In [10]:
#buat nama baru tanpa data pada kolom yes dan no bernilai 0
dataku2 = dataku[(dataku['yes']==1) | (dataku['no']==1)]

In [ ]:
dataku2.shape

(1243, 9)

setelah pengurangan data, tersisa 1243 baris data yang ada

In [11]:
#lihat info data
dataku2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1243 entries, 2 to 8071
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   1243 non-null   object
 1   experiment   1243 non-null   object
 2   date         1243 non-null   object
 3   hour         1243 non-null   int64 
 4   device_make  1243 non-null   object
 5   platform_os  1243 non-null   int64 
 6   browser      1243 non-null   object
 7   yes          1243 non-null   int64 
 8   no           1243 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 97.1+ KB


In [12]:
#lihat apakah ada data duplikat pada id user
dataku2.duplicated().sum() 

0

In [13]:
#dataku2.describe(include='all')
dataku2.describe()

,hour,platform_os,yes,no
count,1243.000000,1243.000000,1243.000000,1243.000000
mean,11.520515,5.986323,0.460177,0.539823
std,5.765704,0.116191,0.498612,0.498612
min,0.000000,5.000000,0.000000,0.000000
25%,7.000000,6.000000,0.000000,0.000000
50%,13.000000,6.000000,0.000000,1.000000
75%,15.000000,6.000000,1.000000,1.000000
max,23.000000,6.000000,1.000000,1.000000


In [14]:
#statistik deskriptif kolom experiment
dataku2['experiment'].describe()

count        1243
unique          2
top       exposed
freq          657
Name: experiment, dtype: object

In [15]:
#lihat detail kolom experiment
dataku2['experiment'].value_counts()

exposed    657
control    586
Name: experiment, dtype: int64

In [38]:
#cari jumlah user berdasarkan grup experiment
control = dataku2[dataku2['experiment'] == 'control']
exposed = dataku2[dataku2['experiment'] == 'exposed']

jml_control = control['yes'].count()
jml_exposed = exposed['yes'].count()

In [39]:
#cari jumlah user yang berikan respon yes dan no
yes_control = control['yes'].sum()
yes_exposed = exposed['yes'].sum()

no_control = jml_control - yes_control
no_exposed = jml_exposed - yes_exposed

In [40]:
jml = np.array([[no_control, no_exposed], [yes_control, yes_exposed]])

print(jml)

[[322 349]
 [264 308]]


**Inference Analysis**

Menggunakan chi-squared test



*   H0 : Tidak terdapat perbedaan yang signifikan antara tingkat keberhasilan periklanan pada iklan biasa dan iklan interaktif.
*   H1: Terdapat perbedaan yang signifikan antara tingkat keberhasilan periklanan pada kelompok biasa dan interaktif.
*   Menggunakan nilai alpha = 0,05
*   Jika nilai p <= alpha, H1 berarti jawabannya
*   Jika nilai p > alpha, H0 berarti jawabannya





In [16]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

In [41]:
#chi test
alpha = 0.05

print(f"""nilai chi2 : {chi2}

nilai-p : {p}

{'Terima H1 dengan senang hati' if p <= alpha else 'Terima H0 dengan lapang dada'}""")

nilai chi2 : 0.5560768104229136

nilai-p : 0.3465426444731173

Terima H0 dengan lapang dada


**Kesimpulan**

Kesimpulan yang didapatkan yaitu menerima **H0 (Tidak terdapat perbedaan yang signifikan antara tingkat keberhasilan periklanan pada iklan biasa dan iklan interaktif)**. Perusahaan dapat memilih salah satu jenis iklan sesuai dengan kemampuan keuangan, karena iklan interaktif jauh lebih mahal

**Export data**

In [ ]:
output_file_path_excel = 'D:\data\DataScienceProcessed.xlsx'
output_file_path_csv = 'D:\data\DataScienceProcessed.csv'
dataku2.to_excel(output_file_path_excel, index=False)
dataku2.to_csv(output_file_path_csv, sep='$', index=False) #csv separator used = '$'

print(dataku2)

                                auction_id experiment        date  hour  \
2     0016d14a-ae18-4a02-a204-6ba53b52f2ed    exposed  2020-07-05     2   
16    008aafdf-deef-4482-8fec-d98e3da054da    exposed  2020-07-04    16   
20    00a1384a-5118-4d1b-925b-6cdada50318d    exposed  2020-07-06     8   
23    00b6fadb-10bd-49e3-a778-290da82f7a8d    control  2020-07-08     4   
27    00ebf4a8-060f-4b99-93ac-c62724399483    control  2020-07-03    15   
...                                    ...        ...         ...   ...   
8059  ffa08ff9-a132-4051-aef5-01a9c79367bc    exposed  2020-07-05    21   
8063  ffb176df-ecd2-45d3-b05f-05b173a093a7    exposed  2020-07-04     1   
8064  ffb79718-6f25-4896-b6b3-e58b80a6e147    control  2020-07-09     7   
8069  ffca1153-c182-4f32-9e90-2a6008417497    control  2020-07-10    16   
8071  ffdfdc09-48c7-4bfb-80f8-ec1eb633602b    exposed  2020-07-03     4   

             device_make  platform_os                browser  yes  no  
2                  E5823   